<a href="https://colab.research.google.com/github/RiyadAhmedCS/CSNEA2022/blob/main/model_128x128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###times
######w/ xla + mixed precision : 3m10s
######w/ xla : 2m40s
######w/o xla + mixed precision: 2m41s

Code

In [2]:
import os
os.environ["TF_CCP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [3]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.optimizer.set_jit(True)

In [ ]:
#Add code here to download new datasets

In [4]:
dataset = keras.preprocessing.image_dataset_from_directory(directory ="/content/drive/MyDrive/nea_files/newimgs128",
                                                           label_mode=None,
                                                           image_size=(128, 128),
                                                           batch_size=32,
                                                           shuffle=True
                                                           
                                                           ).map(lambda x: x/255)

Found 250 files belonging to 1 classes.


In [20]:
#Code to initialise model

# discriminator = keras.Sequential(
#     [
#         keras.Input(shape=(128, 128, 3)), 
#         layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
#         layers.LeakyReLU(0.2),
#         layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
#         layers.LeakyReLU(0.2),
#         layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),
#         layers.LeakyReLU(0.2),
#         layers.Conv2D(512, kernel_size=4, strides=2, padding="same"),
#         layers.LeakyReLU(0.2),
#         layers.Flatten(),
#         layers.Dropout(0.2),
#         layers.Dense(1, activation="sigmoid")
#         #Sigmoid use to determine if image real or fake
#     ]
# )

#Load model that has been trained

discriminator = tf.keras.models.load_model('/content/drive/MyDrive/nea_files/model_Discriminator')
print(discriminator.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 64, 64, 64)        3136      
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 64, 64, 64)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 128)       131200    
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 256)       524544    
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                                 
 conv2d_8 (Conv2D)           (None, 8, 8, 512)        

In [21]:
#Code to initialise model

# latent_dim = 128

# generator = keras.Sequential(
#     [
#         layers.Input(shape=(latent_dim,)),
#         layers.Dense(8*8*128),
#         layers.Reshape((8,8,128)),
#         layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
#         layers.LeakyReLU(0.2),
#         layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
#         layers.LeakyReLU(0.2),
#         layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
#         layers.LeakyReLU(0.2),
#         layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
#         layers.LeakyReLU(0.2),
#         layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
#     ]
# )

#Load model that has been trained

generator = tf.keras.models.load_model('/content/drive/MyDrive/nea_files/model_Generator/')
print(generator.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8192)              1056768   
                                                                 
 reshape_1 (Reshape)         (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 16, 16, 128)      262272    
 ranspose)                                                       
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_5 (Conv2DT  (None, 32, 32, 256)      524544    
 ranspose)                                                       
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 32, 32, 256)      

In [18]:
opt_gen = keras.optimizers.Adam(1e-4)
opt_disc = keras.optimizers.Adam(1e-4)
loss_fn = keras.losses.BinaryCrossentropy()

for epoch in range(1,2):
  for i, real in enumerate(tqdm(dataset)):
    batch_size = real.shape[0]
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    fake = generator(random_latent_vectors)

    if i % 100 == 0:
      img = keras.preprocessing.image.array_to_img(fake[0])
      img.save(f"/content/drive/MyDrive/nea_files/generated_imgs/generated_img{epoch}_{i}.png")
    with tf.GradientTape() as disc_tape:
      loss_disc_real = loss_fn(tf.ones(batch_size, 1), discriminator(real))
      loss_disc_fake = loss_fn(tf.zeros(batch_size, 1), discriminator(fake))
      loss_disc = (loss_disc_real + loss_disc_fake) / 2

    grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
    opt_disc.apply_gradients(
        zip(grads, discriminator.trainable_weights)
        )
    
    with tf.GradientTape() as gen_tape:
      fake = generator(random_latent_vectors)
      output = discriminator(fake)
      loss_gen = loss_fn(tf.ones(batch_size, 1), output)

    grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
    opt_gen.apply_gradients(
        zip(grads, generator.trainable_weights)
        )
    


100%|██████████| 8/8 [01:21<00:00, 10.24s/it]


In [19]:
# Save Model

generator.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(1e-4),
    metrics=["accuracy"],
)

discriminator.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(1e-4),
    metrics=["accuracy"],
)

generator.save("/content/drive/MyDrive/nea_files/model_Generator/",
               save_format='tf')

discriminator.save("/content/drive/MyDrive/nea_files/model_Discriminator/",
                   save_format='tf')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/nea_files/model_Generator/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/nea_files/model_Discriminator/assets


In [ ]:
# img = keras.preprocessing.image.array_to_img(fake[0])
# img.save(f"/content/drive/MyDrive/nea_files/generated_imgs/generated_img{epoch}_{i}.png")